In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
data = pd.read_csv('combined_data.csv')
le = LabelEncoder()
label=le.fit_transform(data['Protocol'])
data['Protocol']=label

In [ ]:
data = data.drop(data.columns[0], axis=1)

In [ ]:
data.describe()

,Time,Protocol,Length,Target
count,3.172390e+05,317239.000000,317239.000000,317239.000000
mean,6.745880e+01,14.254222,492.194393,0.945650
std,1.916876e+02,3.108322,952.918346,0.226708
min,3.610000e-07,0.000000,42.000000,0.000000
25%,5.406614e-01,14.000000,66.000000,1.000000
50%,1.879724e+00,14.000000,74.000000,1.000000
75%,5.238090e+01,17.000000,1152.000000,1.000000
max,9.699905e+02,17.000000,44954.000000,1.000000


In [ ]:
data=data.dropna()


In [ ]:
corr_matrix = data.corr()
corr_matrix

C:\Users\91810\AppData\Local\Temp\ipykernel_11032\184011309.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = data.corr()


,Time,Protocol,Length,Target
Time,1.000000,-0.167834,-0.047586,-0.956120
Protocol,-0.167834,1.000000,-0.156231,0.104349
Length,-0.047586,-0.156231,1.000000,0.016479
Target,-0.956120,0.104349,0.016479,1.000000


In [ ]:
X = data[['Time','Protocol','Length']]
y = data['Target']

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)
importances = rf.feature_importances_
indices = sorted(range(len(importances)), key=lambda i: importances[i], reverse=True)

In [ ]:
X=data[['Time','Length']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled


array([[-0.33721991, -0.45984536],
       [-0.33719915, -0.45984536],
       [-0.33694828, -0.45984536],
       ...,
       [-0.33453992,  1.07229252],
       [-0.3345398 ,  1.07229252],
       [-0.33453965,  1.07229252]])

In [ ]:
X = data.iloc[:,:-1].values
X[:,0]=X_scaled[:,0]*-1
X


array([[0.3372199095369218, '192.168.112.128', '49.44.194.34', 14, 54,
        '40492  >  80 [ACK] Seq=1 Ack=1 Win=64008 Len=0'],
       [0.3371991473364919, '192.168.112.128', '49.44.194.34', 14, 54,
        '40510  >  80 [ACK] Seq=1 Ack=1 Win=63936 Len=0'],
       [0.33694828393219456, '192.168.112.128', '152.195.38.76', 14, 54,
        '49232  >  80 [ACK] Seq=1 Ack=1 Win=63812 Len=0'],
       ...,
       [0.3345399221130892, '192.168.112.128', '192.168.112.129', 14,
        1514,
        '49876  >  80 [ACK] Seq=12450561 Ack=1 Win=501 Len=1448 TSval=1204965619 TSecr=777797 [TCP segment of a reassembled PDU]'],
       [0.3345397970135399, '192.168.112.128', '192.168.112.129', 14,
        1514,
        '52492  >  80 [ACK] Seq=10219313 Ack=1 Win=501 Len=1448 TSval=1204965619 TSecr=777797 [TCP segment of a reassembled PDU]'],
       [0.3345396510049417, '192.168.112.128', '192.168.112.129', 14,
        1514,
        '52456  >  80 [PSH, ACK] Seq=10889281 Ack=1 Win=501 Len=1448 TSval=12049

In [ ]:
y


0         0
1         0
2         0
3         0
4         0
         ..
317234    1
317235    1
317236    1
317237    1
317238    1
Name: Target, Length: 317239, dtype: int64

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42,train_size=0.8)
X_train = np.asarray(X_train)
X_test= np.asarray(X_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential()
model.add(layers.GRU(64, input_shape=(28, 28)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 64)                18048     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 18,954
Trainable params: 18,826
Non-trainable params: 128
_________________________________________________________________
None


In [ ]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255.0, X_test/255.0
X_validate, y_validate = X_test[:-10], y_test[:-10]
X_test, y_test = X_test[-10:], y_test[-10:]

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

In [ ]:
model.fit(
    X_train, y_train, validation_data=(X_validate, y_validate), batch_size=64, epochs=10
)

Epoch 1/10
938/938 [==============================] - 18s 18ms/step - loss: 1.3679 - accuracy: 0.5450 - val_loss: 0.9296 - val_accuracy: 0.6944
Epoch 2/10
938/938 [==============================] - 16s 17ms/step - loss: 0.7607 - accuracy: 0.7488 - val_loss: 0.6929 - val_accuracy: 0.7578
Epoch 3/10
938/938 [==============================] - 16s 17ms/step - loss: 0.5199 - accuracy: 0.8326 - val_loss: 0.4292 - val_accuracy: 0.8557
Epoch 4/10
938/938 [==============================] - 16s 17ms/step - loss: 0.3647 - accuracy: 0.8856 - val_loss: 0.3284 - val_accuracy: 0.8944
Epoch 5/10
938/938 [==============================] - 16s 17ms/step - loss: 0.2839 - accuracy: 0.9118 - val_loss: 0.2881 - val_accuracy: 0.9064
Epoch 6/10
938/938 [==============================] - 16s 17ms/step - loss: 0.2377 - accuracy: 0.9261 - val_loss: 0.2129 - val_accuracy: 0.9326
Epoch 7/10
938/938 [==============================] - 15s 16ms/step - loss: 0.2012 - accuracy: 0.9383 - val_loss: 0.1813 - val_accuracy:

In [ ]:
for i in range(10):
    result = tf.argmax(model.predict(tf.expand_dims(X_test[i], 0)), axis=1)
    print(result.numpy(), y_test[i])

1/1 [==============================] - 0s 329ms/step
[7] 7
1/1 [==============================] - 0s 19ms/step
[8] 8
1/1 [==============================] - 0s 20ms/step
[9] 9
1/1 [==============================] - 0s 20ms/step
[0] 0
1/1 [==============================] - 0s 23ms/step
[1] 1
1/1 [==============================] - 0s 24ms/step
[2] 2
1/1 [==============================] - 0s 21ms/step
[3] 3
1/1 [==============================] - 0s 23ms/step
[4] 4
1/1 [==============================] - 0s 21ms/step
[5] 5
1/1 [==============================] - 0s 24ms/step
[6] 6


In [ ]:
model.fit(
    X_train, y_train, validation_data=(X_validate, y_validate), batch_size=32, epochs=25
)

Epoch 1/25
1875/1875 [==============================] - 27s 15ms/step - loss: 0.1521 - accuracy: 0.9521 - val_loss: 0.1839 - val_accuracy: 0.9421
Epoch 2/25
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1306 - accuracy: 0.9595 - val_loss: 0.1355 - val_accuracy: 0.9556
Epoch 3/25
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1183 - accuracy: 0.9632 - val_loss: 0.1384 - val_accuracy: 0.9541
Epoch 4/25
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1077 - accuracy: 0.9663 - val_loss: 0.0806 - val_accuracy: 0.9746
Epoch 5/25
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0992 - accuracy: 0.9694 - val_loss: 0.2718 - val_accuracy: 0.9091
Epoch 6/25
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0933 - accuracy: 0.9712 - val_loss: 0.0826 - val_accuracy: 0.9723
Epoch 7/25
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0872 - accuracy: 0.9728 - val_loss: 0.0733 -